In [44]:
import pandas as pd
from catboost import CatBoostClassifier
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import numpy as np
from sklearn.model_selection import GridSearchCV

In [19]:
data_clean = pd.read_csv('/content/sample_data/clean_data_loan_modelling.csv')

In [21]:
data_clean

,Loan_Status,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Gender_nan,...,Married_Yes,Married_nan,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Self_Employed_nan,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,1,0.0,5849,0.0,146.412162,360.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0,1.0,4583,1508.0,128.000000,360.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1,0.0,3000,0.0,66.000000,360.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1,0.0,2583,2358.0,120.000000,360.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
4,1,0.0,6000,0.0,141.000000,360.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,1,0.0,2900,0.0,71.000000,360.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
610,1,3.0,4106,0.0,40.000000,180.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
611,1,1.0,8072,240.0,253.000000,360.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
612,1,2.0,7583,0.0,187.000000,360.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [26]:
x = data_clean.drop(columns='Loan_Status', index=None)
y = data_clean['Loan_Status']

In [27]:
y

0      1
1      0
2      1
3      1
4      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Loan_Status, Length: 614, dtype: int64

In [28]:
x

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Gender_nan,Married_No,Married_Yes,Married_nan,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Self_Employed_nan,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0.0,5849,0.0,146.412162,360.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1.0,4583,1508.0,128.000000,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,3000,0.0,66.000000,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,2583,2358.0,120.000000,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,6000,0.0,141.000000,360.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,2900,0.0,71.000000,360.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
610,3.0,4106,0.0,40.000000,180.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
611,1.0,8072,240.0,253.000000,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
612,2.0,7583,0.0,187.000000,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [32]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=30)

In [29]:
start_time = time.time()

In [34]:
cb_loan = CatBoostClassifier(n_estimators=100,
                       loss_function='Logloss',
                       learning_rate=0.1,
                       depth=3, 
                        task_type='CPU',
                       random_state=1,
                       verbose=False)

In [35]:
cb_loan.fit(x_train, y_train)

In [36]:
loan_predicted = cb_loan.predict(x_test)

In [39]:
model_rmse = np.sqrt(mse(y_test, loan_predicted))

In [41]:
model_rmse

0.48556395842728817

In [48]:
cb_loan_grid = CatBoostClassifier(n_estimators=100,
                       loss_function='Logloss',
                       learning_rate=0.1,
                        task_type='CPU',
                       random_state=1,
                       verbose=False)

In [49]:
grid_loan = {
    'max_depth' : [3, 4, 5],
    'n_estimators' : [100, 200, 300]
}

In [50]:
grid_cat = GridSearchCV(
    estimator = cb_loan_grid,
    param_grid= grid_loan,
    scoring= 'accuracy',
    cv = 5
)

In [51]:
grid_cat.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x7f1acad57310>,
             param_grid={'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [52]:
loan_predicted = grid_cat.predict(x_test)


In [53]:
model_rmse = np.sqrt(mse(y_test, loan_predicted))


In [54]:
model_rmse

0.4938647983247948